In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from datetime import datetime
import textwrap
import logging
import os
from dotenv import load_dotenv

load_dotenv()

os.makedirs("emails", exist_ok=True)
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logs/email_gen.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

if os.getenv("GEMINI_API_KEY"):
    logger.info("API Key Found")
else:
    logger.error("GEMINI_API_KEY not found")
    raise ValueError("GEMINI_API_KEY not found in environment variables")

2026-02-06 19:59:40,498 - INFO - API Key Found


In [24]:
USER_PROMPT = input("Enter the topic you want to write email about: ")
logger.info(f"User requested email for topic: {USER_PROMPT}")

SYSTEM_PROMPT = """
You are a professional email writer. Write emails that sound natural and human.

RULES:
1. Keep body under 120 words - be concise
2. Write in natural, conversational tone
3. Each sentence should be meaningful
4. No placeholder brackets like [Your Name] - use actual names from context
5. Be direct and professional
"""

2026-02-06 19:59:55,459 - INFO - User requested email for topic: email to a friend asking about how he is 


In [25]:
class Email(BaseModel):
    topic: str = Field(description="Topic in 3-5 words")
    greeting: str = Field(description="Greeting like 'Dear Mr. Smith,' or 'Hi Team,'")
    subject: str = Field(description="Subject line under 10 words")
    body: str = Field(description="Email body under 120 words. Natural human tone. No placeholders.")
    closing: str = Field(description="Sign-off with name, e.g. 'Best regards,\nSougata'")

def format_email(email: Email) -> str:
    body = email.body.strip()
    formatted_lines = []
    current_line = ""
    
    for word in body.split():
        if len(current_line) + len(word) + 1 <= 75:
            current_line += (" " if current_line else "") + word
        else:
            formatted_lines.append(current_line)
            current_line = word
        
        if current_line.endswith(('.', '?', '!')):
            formatted_lines.append(current_line)
            current_line = ""
    
    if current_line:
        formatted_lines.append(current_line)
    
    formatted_body = "\n".join(formatted_lines)
    
    return f"""Subject: {email.subject}

{email.greeting}

{formatted_body}

{email.closing}
"""

def save_email(email: Email, topic: str) -> str:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_topic = "".join(c if c.isalnum() or c in " _-" else "_" for c in topic[:30])
    filename = f"emails/{timestamp}_{safe_topic}.txt"
    
    formatted = format_email(email)
    with open(filename, "w", encoding="utf-8") as f:
        f.write(formatted)
    
    logger.info(f"Email saved to {filename}")
    return filename

In [26]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)
structured_model = model.with_structured_output(Email)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "Write an email about: {topic}")
])

chain = prompt | structured_model

logger.info("Generating email...")
result = chain.invoke({"topic": USER_PROMPT})
logger.info("Email generated successfully")

filename = save_email(result, USER_PROMPT)

print("\n" + "="*60)
print("✉️  GENERATED EMAIL (Ready to Send)")
print("="*60)
print(format_email(result))
print("="*60)
print(f"📁 Saved to: {filename}")

2026-02-06 20:00:06,882 - INFO - Generating email...
2026-02-06 20:00:06,889 - INFO - AFC is enabled with max remote calls: 10.
2026-02-06 20:00:09,541 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2026-02-06 20:00:09,547 - INFO - Email generated successfully
2026-02-06 20:00:09,551 - INFO - Email saved to emails/20260206_200009_email to a friend asking about.txt



✉️  GENERATED EMAIL (Ready to Send)
Subject: How have you been?

Hi Alex,

I know it's been a little while, so I wanted to reach out and see how
you're doing.
I hope everything is going well on your end.
I've been thinking about you and wanted to catch up sometime soon.
Let me know what you've been up to lately.
No pressure at all, just wanted to say hello!

Best,
Sarah

📁 Saved to: emails/20260206_200009_email to a friend asking about.txt
